## Environment Init

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Chat

In [2]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
message = [HumanMessage(content="Translate this sentence from English to French: I love programming.")]
chat.invoke(message)

AIMessage(content="J'adore la programmation.")

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming."),
]
chat.invoke(messages)

In [ ]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=chat)
conversation.invoke("Translate this sentence from English to French: I love programming.")

In [ ]:
conversation.invoke("Translate it to German.")

## Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")
memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context(
    {"input": "im working on better docs for chatbots"},
    {"output": "oh, that sounds like a lot of work"},
)
memory.save_context(
    {"input": "yes, but it's worth the effort"},
    {"output": "agreed, good docs are important!"},
)
memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.load_memory_variables({})

## Conversation

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import (ChatPromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder,SystemMessagePromptTemplate,)
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[  # The `variable_name` here is what must align with memory
        SystemMessagePromptTemplate.from_template("You are a nice chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder; # Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation.invoke({"question": "hi"})

In [ ]:
conversation.invoke({"question": "Translate this sentence from English to French: I love programming."})

In [ ]:
conversation.invoke({"question": "Now translate the sentence to German."})

## Chat Retrieval

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

llm = ChatOpenAI()
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, p)

qa.invoke("How do agents use Task decomposition?")

In [ ]:
qa.invoke("What are the various ways to implement memory to support it?")